In [10]:
# !pip install polars

In [7]:
# !pip install Bio

In [1]:
from Bio import Entrez
Entrez.email = "jing.wang20@nih.gov"
record = Entrez.elink(dbfrom="pubmed", db="pmc", id="10937393")
print(Entrez.read(record))


[{'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'pubmed', 'IdList': ['10937393']}]


In [54]:
img_url

'https://cdn.ncbi.nlm.nih.gov/pmc/blobs/2744/9451509/e1e20aa69ca5/3486_Fig5.jpg'

In [50]:
# pmc_ids = ["9451509"]#, "10864173"]
# df = scrape_pmc_figures_html(pmc_ids)
# df.write_csv("scraped_figures.csv")
# download_scraped_images(df, "img")
import os
import requests
from bs4 import BeautifulSoup

def write_text(soup, text_file_path):
# Step 1: Find the main article body
    main_section = soup.find('section', class_='body main-article-body')
    title_tag = soup.find('h1')  # Adjust if your title is in a different tag
    title = title_tag.get_text(strip=True) if title_tag else "Untitled"
    if main_section:
        # Step 2: Remove figures, tables, images
        for tag in main_section.find_all(['figure', 'table', 'img', 'aside']):
            tag.decompose()
    
        # Step 3: Find and remove the References section
        ref_heading = main_section.find(lambda tag: tag.name in ['h2', 'h3'] and 'reference' in tag.text.strip().lower())
        if ref_heading:
            # Remove the reference heading and all following siblings
            for sibling in list(ref_heading.find_all_next()):
                sibling.decompose()
            ref_heading.decompose()
    
        # Step 4: Extract cleaned text
        main_text = main_section.get_text(separator='\n', strip=True)
        # Combine title and main content
        full_text = f"{title}{main_text}"
        with open(text_file_path, "w") as text_file:
            text_file.write(full_text)
        print("save text to file")
        return True
        # print(main_text)
    else:
        print("Main article body not found.")
        return False
        
def image_download(soup, save_folder):
    # Find all image tags inside <p class="img-box ...">
    img_tags = soup.select('p.img-box img')
    # Download and save images
    for i, img in enumerate(img_tags, 1):
        img_url = img.get('src')
        if img_url:
            try:
                response = requests.get(img_url)
                response.raise_for_status()  # raise error for bad status
                # Extract filename from URL
                filename = os.path.basename(img_url)
                file_path = os.path.join(save_folder, filename)
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded {i}: {file_path}")
            except Exception as e:
                print(f"Failed to download {img_url}: {e}")

pmcid = "10476922" #"9451509"
url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{pmcid}/"
response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to access PMC article: {url}")

soup = BeautifulSoup(response.content, "html.parser")
save_folder = "."
img_folder = save_folder + "/images"
text_folder = save_folder + "/text"
text_file_path = text_folder + "/" + pmcid 
os.makedirs(text_folder, exist_ok=True)
os.makedirs(img_folder, exist_ok=True)
if not write_text(soup, text_file_path):
    print(pmcid)
soup = BeautifulSoup(response.content, "html.parser")
image_download(soup, img_folder)


save text to file


In [53]:
soup = BeautifulSoup(response.content, "html.parser")
image_download(soup, img_folder)

Downloaded 1: ./images/SMJ-64-517-g001.jpg
Downloaded 2: ./images/SMJ-64-517-g002.jpg


In [52]:
img_tags = soup.select('p.img-box img')
img_tags

[]

In [59]:
img_url.split('/')[6]

'9451509'